In [7]:
import pickle

with open('polizeiberichte.pkl', 'rb') as f:
    data = pickle.load(f, encoding="utf8")

In [8]:
from collections import defaultdict

def extract_verb_ver1(corpus):
    sentence_extract = defaultdict(dict)
    for i, sentence in enumerate(corpus):
        first_propn = ""
        second_propn = ""
        verb = ""
        pron = ""
        for word in sentence:
            if word["upostag"] == "PROPN":
                if first_propn == "":
                    first_propn = word["form"]
                elif second_propn == "":
                    second_propn = word["form"]
            if word["upostag"] == "VERB":
                if verb == "" and first_propn != "" and second_propn == "":
                    verb = word["form"]
            if word["upostag"] == "PRON":
                if verb != "" and first_propn != "" and second_propn != "" and pron == "":
                    pron = word["form"]
        if first_propn != "" and second_propn != "" and verb != "" and pron != "":
            sentence_extract[i]["first"] = first_propn
            sentence_extract[i]["verb"] = verb
            sentence_extract[i]["second"] = second_propn
            sentence_extract[i]["pron"] = pron
            sentence_extract[i]["sentence"] = [w["form"] for w in sentence]
    return sentence_extract

In [11]:
import spacy
from spacy_conll import ConllFormatter

spacy.prefer_gpu()
nlp = spacy.load('de_core_news_md')

3	0,00001% der Stimmrechte (das entspricht 14 Stimmrechten) sind der Gesellschaft gemäß § 22 Abs. 1, Satz 1, Nr. 2 in Verbindung mit Satz 2 WpHG zuzurechnen.



In [12]:
conllformatter = ConllFormatter(nlp)
nlp.add_pipe(conllformatter, after='parser')

In [13]:
example_text = data[1]["text1"]

In [14]:
doc = nlp(example_text)

conll = doc._.conll

In [15]:
for i in range(0, 2):
    example_text = data[i]["text1"]
    if example_text:
        doc = nlp(example_text)
        conll = doc._.conll
        extraction = extract_verb_ver1(conll)
        if extraction:
            print(extraction)

In [16]:
doc = nlp("Alex sagt chris, dass er essen machen soll")
conll = doc._.conll
print(conll)
#extract_verb_ver1(conll)


[[OrderedDict([('id', 1), ('form', 'Alex'), ('lemma', 'Alex'), ('upostag', 'PROPN'), ('xpostag', 'NE'), ('feats', '_'), ('head', 2), ('deprel', 'sb'), ('deps', '_'), ('misc', '_')]), OrderedDict([('id', 2), ('form', 'sagt'), ('lemma', 'sagen'), ('upostag', 'VERB'), ('xpostag', 'VVFIN'), ('feats', 'Mood=ind|VerbForm=fin'), ('head', 0), ('deprel', 'ROOT'), ('deps', '_'), ('misc', '_')]), OrderedDict([('id', 3), ('form', 'chris'), ('lemma', 'chris'), ('upostag', 'PROPN'), ('xpostag', 'NE'), ('feats', '_'), ('head', 2), ('deprel', 'oa'), ('deps', '_'), ('misc', 'SpaceAfter=No')]), OrderedDict([('id', 4), ('form', ','), ('lemma', ','), ('upostag', 'PUNCT'), ('xpostag', '$,'), ('feats', 'PunctType=comm'), ('head', 2), ('deprel', 'punct'), ('deps', '_'), ('misc', '_')]), OrderedDict([('id', 5), ('form', 'dass'), ('lemma', 'dass'), ('upostag', 'SCONJ'), ('xpostag', 'KOUS'), ('feats', '_'), ('head', 9), ('deprel', 'cp'), ('deps', '_'), ('misc', '_')]), OrderedDict([('id', 6), ('form', 'er'), ('

In [17]:
def _get_nearest_propn(conll, index):
    distance_dict = {}
    left = None
    right = None
    for i in range(index - 1, -1, -1):
        if conll[i]["upostag"] == "PROPN":
            left = (index - i)*-1
            break
    for i in range(index + 1, len(conll)):
        if conll[i]["upostag"] == "PROPN":
            right = i - index
            break
    if left and right:
        if abs(left + right) < 3:
            return (left, right)
   

def get_verb_dict(corpus):
    sentence_extract = defaultdict(dict)
    verb_dict = defaultdict(dict)
    for i, sentence in enumerate(corpus):
        for j, word in enumerate(sentence):
            if word["upostag"] == "VERB":
                nearest_propn = _get_nearest_propn(sentence, j)
                if nearest_propn:
                    verb_dict[i][word["form"]] = (sentence[j+nearest_propn[0]]["form"], sentence[j+nearest_propn[1]]["form"])
    return verb_dict

In [18]:
get_verb_dict(conll)

defaultdict(dict, {0: {'sagt': ('Alex', 'chris')}})

In [11]:
for i in range(0, 200):
    example_text = data[i]["text1"]
    if example_text:
        doc = nlp(example_text)
        conll = doc._.conll
        extraction = get_verb_dict(conll)
        if extraction:
            print(i, extraction)

25 defaultdict(<class 'dict'>, {2: {'erfasste': ('Allee', 'BMW')}})
29 defaultdict(<class 'dict'>, {1: {'bog': ('Heinrich-Mann-Straße', 'Hermann-Hesse-Straße')}})
128 defaultdict(<class 'dict'>, {1: {'erhob': ('Uhlandstraße', 'Heil')}})
135 defaultdict(<class 'dict'>, {12: {'bemerkten': ('Schmidstraße', 'VW')}})
153 defaultdict(<class 'dict'>, {1: {'überholt': ('Damms', 'Rhenaniastraße')}})
159 defaultdict(<class 'dict'>, {2: {'bremsen': ('Renault', 'VW')}})


In [12]:
data[25]["text1"]

'Die Polizei Berlin sucht dringend Zeugen zu einem Verkehrsunfall vom vergangenen Samstag, 23. August 2014, in Marzahn. Wie berichtet, fuhren ein „Nissan“ und ein „BMW“ gegen 21.50 Uhr an der Kreuzung Landsberger Allee/Blumberger Damm in Fahrtrichtung stadtauswärts bei „Grün“ sehr rasant mit aufheulenden Motorengeräuschen an. Vor der Hausnummer 563 in der Landsberger Allee erfasste der „BMW“ eine Radfahrerin, die nach den bisherigen Erkenntnissen bei „Rot“ die Fußgängerampel überquerte. Bei dem Unfall erlitt die 27-jährige Radfahrerin lebensbedrohliche Verletzungen und wird derzeit weiterhin intensivmedizinisch betreut.'

In [23]:
with open('deu_news_2015_3M-sentences.txt', 'rb') as f:
    f.seek(1)
    ger_sent = pickle.load(f, encoding="utf8")



UnpicklingError: invalid load key, '\x09'.

In [2]:
file1 = open('deu_news_2015_3M-sentences.txt', 'r') 
Lines = file1.readlines() 
print(Lines)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
ger_lines = Lines[:10000]

In [19]:
for i in range(0, 200):
    example_text = ger_lines[i]
    if example_text:
        doc = nlp(example_text)
        conll = doc._.conll
        extraction = get_verb_dict(conll)
        if extraction:
            print(i, extraction)

11 defaultdict(<class 'dict'>, {1: {'kappt': ('Serbien', 'Kroatien')}})
28 defaultdict(<class 'dict'>, {0: {'gibt': ('Litauen', 'Russland')}})
38 defaultdict(<class 'dict'>, {1: {'filmt': ('Altaussee', 'Bond-Crew')}})
46 defaultdict(<class 'dict'>, {0: {'gibt': ('Liverpool', 'Boxing')}})
54 defaultdict(<class 'dict'>, {1: {'umfassen': ('DGAP', 'Corporate')}})
68 defaultdict(<class 'dict'>, {1: {'identifiziert': ('Interpol', 'Irak')}})
76 defaultdict(<class 'dict'>, {2: {'gehen': ('ots', 'Deutschland')}})
77 defaultdict(<class 'dict'>, {0: {'übermittelt': ('News/Finanznachricht', 'DGAP')}})
87 defaultdict(<class 'dict'>, {1: {'umfassen': ('DGAP', 'Corporate')}})
89 defaultdict(<class 'dict'>, {2: {'entgleisen': ('Gerd', '@balli')}})
93 defaultdict(<class 'dict'>, {4: {'beschlagnahmen': ('Hamburg', 'Ulrich')}})
110 defaultdict(<class 'dict'>, {1: {'einzuschüchtern': ('Russlands', 'Wladimir')}})
113 defaultdict(<class 'dict'>, {2: {'schließt': ('IWF', 'Athens')}})
119 defaultdict(<class '

In [24]:
print(Lines[119])

120	01498 USA schmieden Bündnis gegen China und wollen Russland ausbootenWashington bemüht sich um eine antichinesische Koalition in Südostasien, schreibt die "Nesawissimaja Gaseta" am Dienstag.

